In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('/Users/Ash/Desktop/BOOTCAMP/Class/MOD 19 MODULE WORK/HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


generate a list of categorical variable names

In [2]:
#use our variable list to perform the one-hot encoding once, rather than for each individual variable. 
#To generate our variable list, we'll use Pandas Dataframe.dtypes property.

# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [3]:
#variable names now seperated, start preprocess our data starting with the one-hot encoding of the categorical data

In [4]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [5]:
#According to the nunique method, none of the categorical variables have more than 10 unique values, which means we're ready to encode.

In [6]:
#The only difference from our single variable example is we need to pass our attrition_cat variable list
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [7]:
#To replace these columns, we'll use a combination of Pandas' merge and drop methods

# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [8]:
#separate our features and target as well as perform our training/test split

# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
#build our StandardScalerobject and standardize the numerical features

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

our data is preprocessed and separated and ready for modelling.

In [10]:
#create deep learning model

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


compile our model and define the loss and accuracy metrics

In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

train the model

In [12]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
35/35 [==============================] - 1s 841us/step - loss: 956.2372 - accuracy: 0.1724
Epoch 2/100
35/35 [==============================] - 0s 731us/step - loss: 418.4508 - accuracy: 0.2096
Epoch 3/100
35/35 [==============================] - 0s 707us/step - loss: 145.8444 - accuracy: 0.4510
Epoch 4/100
35/35 [==============================] - 0s 651us/step - loss: 21.1108 - accuracy: 0.7486
Epoch 5/100
35/35 [==============================] - 0s 745us/step - loss: 13.8829 - accuracy: 0.7668
Epoch 6/100
35/35 [==============================] - 0s 854us/step - loss: 10.7271 - accuracy: 0.7704
Epoch 7/100
35/35 [==============================] - 0s 1ms/step - loss: 8.3338 - accuracy: 0.7750
Epoch 8/100
35/35 [==============================] - 0s 780us/step - loss: 7.5226 - accuracy: 0.7895
Epoch 9/100
35/35 [==============================] - 0s 722us/step - loss: 6.4669 - accuracy: 0.7895
Epoch 10/100
35/35 [==============================] - 0s 835us/step - loss: 5.6025 -

35/35 [==============================] - 0s 836us/step - loss: 0.7449 - accuracy: 0.8221
Epoch 82/100
35/35 [==============================] - 0s 773us/step - loss: 0.9335 - accuracy: 0.8149
Epoch 83/100
35/35 [==============================] - 0s 765us/step - loss: 1.1030 - accuracy: 0.7777
Epoch 84/100
35/35 [==============================] - 0s 770us/step - loss: 0.9062 - accuracy: 0.8158
Epoch 85/100
35/35 [==============================] - 0s 978us/step - loss: 0.6769 - accuracy: 0.8167
Epoch 86/100
35/35 [==============================] - 0s 960us/step - loss: 0.7243 - accuracy: 0.8185
Epoch 87/100
35/35 [==============================] - 0s 770us/step - loss: 0.7329 - accuracy: 0.8221
Epoch 88/100
35/35 [==============================] - 0s 711us/step - loss: 0.7802 - accuracy: 0.8031
Epoch 89/100
35/35 [==============================] - 0s 854us/step - loss: 0.8224 - accuracy: 0.8113
Epoch 90/100
35/35 [==============================] - 0s 951us/step - loss: 0.7072 - accuracy: 

In [13]:
#deep learning model is properly trained, we can evaluate the model's performance by testing its predictive capabilities on our testing dataset.

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 1.9122 - accuracy: 0.8234
Loss: 1.9122031927108765, Accuracy: 0.823369562625885


In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

Once we have defined the file structure and filepath, we need to create a callback object for our deep learning model

Bringing it all together, we can compile, train, and evaluate our deep learning model

In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
35/35 [==============================] - 0s 881us/step - loss: 0.1697 - accuracy: 0.9419
Epoch 2/100
35/35 [==============================] - 0s 951us/step - loss: 0.1664 - accuracy: 0.9419
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.1474 - accuracy: 0.9688
Epoch 00003: saving model to checkpoints/weights.03.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.1653 - accuracy: 0.9410
Epoch 4/100
35/35 [==============================] - 0s 895us/step - loss: 0.1646 - accuracy: 0.9428
Epoch 5/100
35/35 [==============================] - 0s 834us/step - loss: 0.1642 - accuracy: 0.9446
Epoch 6/100
 1/35 [..............................] - ETA: 0s - loss: 0.1436 - accuracy: 0.9375
Epoch 00006: saving model to checkpoints/weights.06.hdf5
35/35 [==============================] - 0s 973us/step - loss: 0.1637 - accuracy: 0.9419
Epoch 7/100
35/35 [==============================] - 0s 773us/step - loss: 0.1634 - accuracy: 0.9437
Epoch 8/100
35/

Epoch 56/100
35/35 [==============================] - 0s 885us/step - loss: 0.1329 - accuracy: 0.9592
Epoch 57/100
35/35 [==============================] - 0s 938us/step - loss: 0.1331 - accuracy: 0.9574
Epoch 58/100
 1/35 [..............................] - ETA: 0s - loss: 0.0374 - accuracy: 1.0000
Epoch 00058: saving model to checkpoints/weights.58.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.1330 - accuracy: 0.9546
Epoch 59/100
35/35 [==============================] - 0s 787us/step - loss: 0.1327 - accuracy: 0.9601
Epoch 60/100
 1/35 [..............................] - ETA: 0s - loss: 0.1263 - accuracy: 0.9688
Epoch 00060: saving model to checkpoints/weights.60.hdf5
35/35 [==============================] - 0s 835us/step - loss: 0.1324 - accuracy: 0.9564
Epoch 61/100
35/35 [==============================] - 0s 874us/step - loss: 0.1308 - accuracy: 0.9564
Epoch 62/100
35/35 [==============================] - 0s 974us/step - loss: 0.1308 - accuracy: 0.9583
Epoch 63

we have created our trained model within the Python session, as well as a folder of checkpoints we can use to restore previous model weights

To test this functionality, let's define another deep learning model, but restore the weights using the checkpoints rather than training the model

In [20]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Train the model
fit_model = nn_new.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
35/35 [==============================] - 0s 613us/step - loss: 0.9232 - accuracy: 0.3149
Epoch 2/100
35/35 [==============================] - 0s 761us/step - loss: 0.6254 - accuracy: 0.6751
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.5235 - accuracy: 0.8125
Epoch 00003: saving model to checkpoints/weights.03.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.4978 - accuracy: 0.8212
Epoch 4/100
35/35 [==============================] - 0s 882us/step - loss: 0.4331 - accuracy: 0.8412
Epoch 5/100
35/35 [==============================] - 0s 892us/step - loss: 0.3976 - accuracy: 0.8457
Epoch 6/100
 1/35 [..............................] - ETA: 0s - loss: 0.3400 - accuracy: 0.8438
Epoch 00006: saving model to checkpoints/weights.06.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.3710 - accuracy: 0.8557
Epoch 7/100
35/35 [==============================] - 0s 1ms/step - loss: 0.3504 - accuracy: 0.8711
Epoch 8/100
35/35 [

Epoch 56/100
35/35 [==============================] - 0s 640us/step - loss: 0.0818 - accuracy: 0.9819
Epoch 57/100
35/35 [==============================] - 0s 773us/step - loss: 0.0788 - accuracy: 0.9809
Epoch 58/100
 1/35 [..............................] - ETA: 0s - loss: 0.0665 - accuracy: 1.0000
Epoch 00058: saving model to checkpoints/weights.58.hdf5
35/35 [==============================] - 0s 946us/step - loss: 0.0755 - accuracy: 0.9819
Epoch 59/100
35/35 [==============================] - 0s 856us/step - loss: 0.0728 - accuracy: 0.9837
Epoch 60/100
 1/35 [..............................] - ETA: 0s - loss: 0.0744 - accuracy: 1.0000
Epoch 00060: saving model to checkpoints/weights.60.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.0702 - accuracy: 0.9855
Epoch 61/100
35/35 [==============================] - 0s 823us/step - loss: 0.0689 - accuracy: 0.9846
Epoch 62/100
35/35 [==============================] - 0s 677us/step - loss: 0.0657 - accuracy: 0.9864
Epoch 63